# 1. Limpieza de los datos

In [ ]:
!pip install pandasql

In [30]:
import pandasql as ps
import pandas as pd
import numpy as np

In [3]:
#Lectura
b_metadata = pd.read_csv('./building_metadata.csv')
weather_train = pd.read_csv('./weather_train.csv', parse_dates=True, index_col='timestamp')
weather_test = pd.read_csv('./weather_test.csv', parse_dates=True, index_col='timestamp')
meters_train = pd.read_csv('./train.csv')
meters_test = pd.read_csv('./test.csv')

## 1.1 Previsualización y Organización de las Tablas 

### 1.1.1 Metadatos de los edificios

In [ ]:
# Metadatos de los edificios
b_metadata.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [4]:
query = ps.sqldf("select primary_use, count(primary_use) from b_metadata group by primary_use")

In [5]:
query

,primary_use,count(primary_use)
0,Education,549
1,Entertainment/public assembly,184
2,Food sales and service,5
3,Healthcare,23
4,Lodging/residential,147
5,Manufacturing/industrial,12
6,Office,279
7,Other,25
8,Parking,22
9,Public services,156


Por fines de memoria RAM disponible, decidimos trabajar únicamente con los edificios residenciales, así que extraemos sus metadatos.

In [6]:
residential_meta = ps.sqldf("select * from b_metadata where primary_use = 'Lodging/residential'")

In [7]:
residential_meta.isna().value_counts()

site_id  building_id  primary_use  square_feet  year_built  floor_count
False    False        False        False        False       True           71
                                                True        True           58
                                                False       False          14
                                                True        False           4
dtype: int64

Vemos que en la mayor parte de los registros, aunque no toda, se cuenta con la información de número de pisos. Sin embargo todos contienen la información del área en pies cuadrados, la cual corresponde al área bruta del edificio. 
Resulta intuitivo pensar que el área bruta depende proporcionalmente del número de pisos que tiene el edificio en cuestión. De tal manera, no se considerará la información del número de pisos.

Para el caso del año en que se construyó el edificio en cuestión vemos que un 57% de los registros si contienen dicha información y el otro 43% no De tal forma es posible conservar dicha información y los valores nulos mandarlos a 0. 
Es también razonable pensar que por más antiguo que sea un edificio, es normal que reciban ciertas remodelaciones en las distintas instalaciones, sin querer decir que por ser más antiguos tendrán un consumo distinto a los más modernos. Por lo mismo decidimos dejar atrás también dicha información.

In [8]:
residential_meta = residential_meta.drop( columns=['floor_count', 'year_built'])

Comprobamos como luce ahora el dataframe con los metadatos

In [9]:
residential_meta.head() 

,site_id,building_id,primary_use,square_feet
0,0,6,Lodging/residential,27926
1,0,12,Lodging/residential,37100
2,0,27,Lodging/residential,59200
3,0,33,Lodging/residential,38686
4,0,34,Lodging/residential,42731


In [11]:
residential_meta.to_csv('./residential_metadata.csv')

### 1.1.2 Datos del clima de entrenamiento



In [12]:
weather_train #previsualización de la tabla

,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
timestamp,,,,,,,,
2016-01-01 00:00:00,0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2016-01-01 01:00:00,0,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2016-01-01 02:00:00,0,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
2016-01-01 03:00:00,0,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
2016-01-01 04:00:00,0,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
...,...,...,...,...,...,...,...,...
2016-12-31 19:00:00,15,3.0,NaN,-8.0,NaN,NaN,180.0,5.7
2016-12-31 20:00:00,15,2.8,2.0,-8.9,NaN,1007.4,180.0,7.7
2016-12-31 21:00:00,15,2.8,NaN,-7.2,NaN,1007.5,180.0,5.1


Vemos qué tantos valores nulos tenemos por cada uno de los sitios y nos enfrentamos al problema de cómo llenar los valores faltantes.

El siguiente análisis pretende resolver dicha interrogante para tomar decisiones al respecto. El análisis se enfoca principalmente en las variables climáticas de nubosidad y precipitación, pues son las más ausentes en los datos como se comprueba a continuación.

In [13]:
sitios = ps.sqldf('select site_id from weather_train group by site_id') #obtenemos un df con los sitios

In [14]:
for sitio in sitios.site_id:
    local_weather = ps.sqldf("select * from weather_train where site_id = {}".format(sitio))
    print('-'*50)
    print('Sitio {}\n'.format(sitio))
    for attribute in local_weather.columns:
        try:
            percentage = int(local_weather[attribute].isna().value_counts().loc[False])*100/local_weather.shape[0]
            print('\n Para {} SE TIENE el {:.2f}% de los datos.'.format(attribute, percentage))
            try:
                zeros = int(local_weather[attribute].value_counts().loc[0.0])*100/local_weather[attribute].isna().value_counts().loc[False]
                print('El {:.2f}% de los valores existentes son 0.'.format(zeros))
                if attribute == 'precip_depth_1_hr' or attribute == 'cloud_coverage':
                    print('Probabilidad de que el valor faltante sea una falla y no 0 es: {:.2f}.\n'.format(zeros/percentage))
            except:
                print('No tiene valores en 0.')
                if attribute == 'precip_depth_1_hr' or attribute == 'cloud_coverage':
                    print('El valor faltante es  probablemente 0.')
        except:
            percentage = int(local_weather[attribute].isna().value_counts().loc[True])*100/8784
            print('\n {} esta VACIO.'.format(attribute))
            print('No hay medidor.')


--------------------------------------------------
Sitio 0


 Para timestamp SE TIENE el 100.00% de los datos.
No tiene valores en 0.

 Para site_id SE TIENE el 100.00% de los datos.
El 100.00% de los valores existentes son 0.

 Para air_temperature SE TIENE el 99.97% de los datos.
No tiene valores en 0.

 Para cloud_coverage SE TIENE el 56.40% de los datos.
El 17.76% de los valores existentes son 0.
Probabilidad de que el valor faltante sea una falla y no 0 es: 0.31.


 Para dew_temperature SE TIENE el 99.97% de los datos.
El 0.30% de los valores existentes son 0.

 Para precip_depth_1_hr SE TIENE el 99.99% de los datos.
El 90.49% de los valores existentes son 0.
Probabilidad de que el valor faltante sea una falla y no 0 es: 0.91.


 Para sea_level_pressure SE TIENE el 99.03% de los datos.
No tiene valores en 0.

 Para wind_direction SE TIENE el 97.15% de los datos.
El 12.01% de los valores existentes son 0.

 Para wind_speed SE TIENE el 100.00% de los datos.
El 11.67% de los valores 

--------------------------------------------------
Sitio 9


 Para timestamp SE TIENE el 100.00% de los datos.
No tiene valores en 0.

 Para site_id SE TIENE el 100.00% de los datos.
No tiene valores en 0.

 Para air_temperature SE TIENE el 99.94% de los datos.
El 0.07% de los valores existentes son 0.

 Para cloud_coverage SE TIENE el 60.62% de los datos.
El 79.26% de los valores existentes son 0.
Probabilidad de que el valor faltante sea una falla y no 0 es: 1.31.


 Para dew_temperature SE TIENE el 99.92% de los datos.
El 1.11% de los valores existentes son 0.

 Para precip_depth_1_hr SE TIENE el 99.92% de los datos.
El 91.55% de los valores existentes son 0.
Probabilidad de que el valor faltante sea una falla y no 0 es: 0.92.


 Para sea_level_pressure SE TIENE el 97.28% de los datos.
No tiene valores en 0.

 Para wind_direction SE TIENE el 70.87% de los datos.
El 33.88% de los valores existentes son 0.

 Para wind_speed SE TIENE el 98.90% de los datos.
El 24.28% de los valores exi

Se observa que en la mayor parte de los sitios hacen falta los datos de covertura de nubes y de precipitación. Debido a que no es enteramente claro el motivo del porqué faltan dichas cantidades de datos ni se indican en la [página de la competencia](https://www.kaggle.com/c/ashrae-energy-prediction/overview), hemos decidido dejar atras dicha información, mientras que para el sitio 5, que no tiene medidor para la presión atmosférica, se rellenarán los valores con 0 y se agregará una columna adicional que indique para cada uno de los sitios si se contaba con dicha información con un 1 y si no, con un 0. 

Los valores faltantes de las demás columnas se completan por medio de interpolaciones.


In [19]:
#se ajusta el dataframe con las fechas para cada uno de los sitios
weather_train = pd.read_csv('./weather_train.csv', parse_dates=True, index_col='timestamp')
weather_train = weather_train.drop( columns=['cloud_coverage', 'precip_depth_1_hr'])

In [20]:
weather_clean_train = pd.DataFrame()
for sitio in sitios.site_id:
    print('Limpiando datos del clima para el sitio {}'.format(sitio))
    local_weather = ps.sqldf("select * from weather_train where site_id = {}".format(sitio)) #read df for each site
    local_weather = local_weather.replace(-1,np.nan)
    local_weather['timestamp'] = pd.to_datetime(local_weather['timestamp'], infer_datetime_format=True) #parse dates
    local_weather = local_weather.set_index('timestamp') #date as index

    local_weather = local_weather.asfreq('H') #set hourly frequency to dataframe

    #atributo que indica si funciona bien o no el medidor de presión
    if sitio == 5 :
        local_weather['pressure_meter'] = np.zeros(local_weather.shape[0]).tolist()
    else:
        local_weather['pressure_meter'] = np.ones(local_weather.shape[0]).tolist()


    #se llenan con 0 los datos del medidor de pressión para el sitio 5.
    if sitio == 5:
        local_weather['sea_level_pressure'] = local_weather['sea_level_pressure'].fillna(0.0)

    #se interpolan el resto de los datos
    local_weather = local_weather.interpolate(method='time', limit_direction='both')
    #se vuelven a juntar los datos del clima limpios para cada uno de los sitios
    weather_clean_train = pd.concat([weather_clean_train, local_weather])

Limpiando datos del clima para el sitio 0
Limpiando datos del clima para el sitio 1
Limpiando datos del clima para el sitio 2
Limpiando datos del clima para el sitio 3
Limpiando datos del clima para el sitio 4
Limpiando datos del clima para el sitio 5
Limpiando datos del clima para el sitio 6
Limpiando datos del clima para el sitio 7
Limpiando datos del clima para el sitio 8
Limpiando datos del clima para el sitio 9
Limpiando datos del clima para el sitio 10
Limpiando datos del clima para el sitio 11
Limpiando datos del clima para el sitio 12
Limpiando datos del clima para el sitio 13
Limpiando datos del clima para el sitio 14
Limpiando datos del clima para el sitio 15


Se comprueba que la limpia de los datos ha sido correcta.

In [21]:
weather_clean_train.isnull().value_counts()

site_id  air_temperature  dew_temperature  sea_level_pressure  wind_direction  wind_speed  pressure_meter
False    False            False            False               False           False       False             140537
dtype: int64

A continuación se transforma la variable de la dirección del viento a categórica. Así mismo se agregan como atributos el día de la semana, hora del día, día del mes y el mes.

In [22]:
def weekday_categorical_int(data, initial_day = 1):
    '''
    Create a categorical value for the number of the day of the week. 1 = monday, 
    2 = tuesday and so on. 
    '''
    day = [initial_day]
    
    for i in range(data.shape[0] - 1):
        if day[i] == 7:
            num_day = 0
        else:
            num_day = day[i]
        day.append(num_day + 1)
    return day

def wind_dir_categorical_int(data):
    wind_dir = data['wind_direction'].astype('float32').tolist()
    for i in range(len(wind_dir)):
        x = wind_dir[i]
        if 22.5 <= x < 67.5:
            wind_dir[i] = 1
        elif 67.5 <= x < 112.5:
            wind_dir[i] = 2
        elif 112.5 <= x < 157.5:
            wind_dir[i] = 3
        elif 157.5 <= x < 202.5:
            wind_dir[i] = 4
        elif 202.5 <= x < 247.5:
            wind_dir[i] = 5
        elif 247.5 <= x < 292.5:
            wind_dir[i] = 6
        elif 292.5 <= x < 337.5:
            wind_dir[i] = 7
        else:
            wind_dir[i] = 0
    return wind_dir

In [23]:
#adding columns with datetime data
weather_clean_train['month'] = [d.month for d in weather_clean_train.index]
weather_clean_train['day'] = [d.day for d in weather_clean_train.index]
weather_clean_train['weekday'] = weekday_categorical_int(weather_clean_train, 5)
weather_clean_train['hour'] = [d.hour for d in weather_clean_train.index]
#overwriting wind_dir column
weather_clean_train['wind_direction'] = wind_dir_categorical_int(weather_clean_train)

Sería posible codificar dichas variables categóricas en One Hot como hicimos anteriormente. Sin embargo, para no incrmenetar excesivamente la dimensionalidad de los datos, decidimos dejarla con la codificación actual (por enteros/labels).

In [24]:
weather_clean_train.head()

,site_id,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,pressure_meter,month,day,weekday,hour
timestamp,,,,,,,,,,,
2016-01-01 00:00:00,0.0,25.0,20.0,1019.7,0,0.0,1.0,1,1,5,0
2016-01-01 01:00:00,0.0,24.4,21.1,1020.2,2,1.5,1.0,1,1,6,1
2016-01-01 02:00:00,0.0,22.8,21.1,1020.2,0,0.0,1.0,1,1,7,2
2016-01-01 03:00:00,0.0,21.1,20.6,1020.1,0,0.0,1.0,1,1,1,3
2016-01-01 04:00:00,0.0,20.0,20.0,1020.0,6,2.6,1.0,1,1,2,4


In [25]:
weather_clean_train.shape

(140537, 11)

In [27]:
weather_clean_train.to_csv('./weather_clean_train.csv')

### 1.1.4 Test and Train Labels

In [1]:
import pandas as pd
import pandasql as ps
import numpy as np

Partiendo de la suposición de que dichos csv están completos, procedemos a la extracción de los datos con los edificios residenciales.

In [43]:
meters_train = pd.read_csv('./train.csv')
residential_meta = pd.read_csv('./residential_metadata.csv')

In [44]:
buildings_id_res = ps.sqldf("select building_id from residential_meta") #id con los edificios residenciales

In [45]:
meters_train.head() #tabla con información sobre medidores

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [47]:
#Filtrado de la información de solo los medidores 0 
meter_0 = ps.sqldf("select * from meters_train where meter = 0")

In [51]:
#Extrayendo únicamente los edificios residenciales
meter_0_res = ps.sqldf("select * from meter_0 where building_id in (select * from buildings_id_res)")

In [59]:
meter_0_res['electricity'] = meter_0_res['meter_reading'].values.tolist()
e_load =  meter_0_res.drop(columns=['meter', 'meter_reading'])
e_load

,building_id,timestamp,electricity
0,6,2016-01-01 00:00:00,0.000
1,12,2016-01-01 00:00:00,0.000
2,27,2016-01-01 00:00:00,0.000
3,33,2016-01-01 00:00:00,0.000
4,34,2016-01-01 00:00:00,0.000
...,...,...,...
1229077,1422,2016-12-31 23:00:00,9.575
1229078,1423,2016-12-31 23:00:00,9.400
1229079,1424,2016-12-31 23:00:00,16.500
1229080,1440,2016-12-31 23:00:00,154.750


In [60]:
e_load.to_csv('./electric_load_residential.csv')

## 1.2 Juntando las tablas

Nos interesa juntar en una sola tabla la información de los medidores, así como los datos del clima para poder insertarlos al sistema como un solo vector de entrada.

In [71]:
residential_meta = pd.read_csv('./residential_metadata.csv')
weather_clean_train = pd.read_csv('./weather_clean_train.csv')
electric_load = pd.read_csv('./electric_load_residential.csv')


In [72]:
residential_meta = residential_meta.drop(columns = ['Unnamed: 0','primary_use'])
electric_load = electric_load.drop(columns = ['Unnamed: 0'])
residential_meta

,site_id,building_id,square_feet
0,0,6,27926
1,0,12,37100
2,0,27,59200
3,0,33,38686
4,0,34,42731
...,...,...,...
142,15,1422,19876
143,15,1423,18277
144,15,1424,24103
145,15,1440,150294


In [73]:
#juntando los datos del medidor con los metadatos del edificio.
input_data = electric_load.merge(residential_meta, on='building_id')

In [74]:
input_data

,building_id,timestamp,electricity,site_id,square_feet
0,6,2016-01-01 00:00:00,0.000,0,27926
1,6,2016-01-01 01:00:00,0.000,0,27926
2,6,2016-01-01 02:00:00,0.000,0,27926
3,6,2016-01-01 03:00:00,0.000,0,27926
4,6,2016-01-01 04:00:00,0.000,0,27926
...,...,...,...,...,...
1229077,647,2016-12-31 19:00:00,19.296,4,86959
1229078,647,2016-12-31 20:00:00,20.078,4,86959
1229079,647,2016-12-31 21:00:00,19.654,4,86959
1229080,647,2016-12-31 22:00:00,19.664,4,86959


In [75]:
#juntando los datos del clima con el resto
input_data = input_data.merge(weather_clean_train, on = ['timestamp', 'site_id'])

In [76]:
input_data

,building_id,timestamp,electricity,site_id,square_feet,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,pressure_meter,month,day,weekday,hour
0,6,2016-01-01 00:00:00,0.000,0,27926,25.0,20.0,1019.7,0,0.0,1.0,1,1,5,0
1,12,2016-01-01 00:00:00,0.000,0,37100,25.0,20.0,1019.7,0,0.0,1.0,1,1,5,0
2,27,2016-01-01 00:00:00,0.000,0,59200,25.0,20.0,1019.7,0,0.0,1.0,1,1,5,0
3,33,2016-01-01 00:00:00,0.000,0,38686,25.0,20.0,1019.7,0,0.0,1.0,1,1,5,0
4,34,2016-01-01 00:00:00,0.000,0,42731,25.0,20.0,1019.7,0,0.0,1.0,1,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229003,1422,2016-06-26 18:00:00,5.300,15,19876,30.6,15.6,1020.3,4,4.1,1.0,6,26,7,18
1229004,1423,2016-06-26 18:00:00,12.725,15,18277,30.6,15.6,1020.3,4,4.1,1.0,6,26,7,18
1229005,1424,2016-06-26 18:00:00,24.250,15,24103,30.6,15.6,1020.3,4,4.1,1.0,6,26,7,18
1229006,1440,2016-06-26 18:00:00,274.600,15,150294,30.6,15.6,1020.3,4,4.1,1.0,6,26,7,18


In [77]:
#se ordenan por número de edificio
input_ordered = ps.sqldf("select * from input_data order by building_id desc, timestamp asc")
input_ordered

,building_id,timestamp,electricity,site_id,square_feet,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,pressure_meter,month,day,weekday,hour
0,1447,2016-01-01 01:00:00,156.650,15,29775,0.0,-2.000000,1019.4,7,4.100000,1.0,1,1,5,1
1,1447,2016-01-01 02:00:00,157.575,15,29775,0.0,-2.000000,1019.4,7,2.600000,1.0,1,1,6,2
2,1447,2016-01-01 03:00:00,154.925,15,29775,0.0,-2.000000,1019.4,0,2.100000,1.0,1,1,7,3
3,1447,2016-01-01 04:00:00,156.075,15,29775,0.0,-2.111111,1019.4,7,2.155556,1.0,1,1,1,4
4,1447,2016-01-01 05:00:00,154.500,15,29775,0.0,-2.222222,1019.4,7,2.211111,1.0,1,1,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229003,6,2016-12-31 19:00:00,264.970,0,27926,22.8,10.000000,1021.7,3,5.700000,1.0,12,31,6,19
1229004,6,2016-12-31 20:00:00,265.379,0,27926,23.3,8.900000,1021.0,4,4.100000,1.0,12,31,7,20
1229005,6,2016-12-31 21:00:00,263.741,0,27926,23.3,10.000000,1021.1,3,4.100000,1.0,12,31,1,21
1229006,6,2016-12-31 22:00:00,258.827,0,27926,22.8,10.000000,1021.1,4,3.100000,1.0,12,31,2,22


In [79]:
input_ordered.to_csv('./data_ordered.csv')